Revison of class 1 to 4 

## Importsnt topics disscussed were 
1 Groq batches 

2 Neural Networks 

3 Frontier Models evaluation

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [2]:
LITE_MODE = True
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

In [3]:
from pricer.items import Item
from pricer.evaluator import evaluate
import numpy as np
from tqdm import tqdm

c:\Users\vijib\projects\llm_eng_manual\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train, val, test = Item.from_hub(dataset)
print(f"Loaded {len(train)} items")

Loaded 20000 items


In [5]:
## First we start from groq batches, define the syntax 
from groq import Groq
import os
groq = Groq(api_key = os.environ.get("GROQ_API_KEY"))
model = "gpt-oss-120b"

SYSTEM_PROMPT = """Create a concise description of a product. Respond only in this format. Do not include part numbers. 
Title: Rewritten short precise title
Category: eg Electronics
Brand: Brand name
Description: 1 sentence description
Details: 1 sentence on features
"""
items = train+test+val
for index, item in enumerate(items):
    item.id = index




In [ ]:
## we need to create jsonl files to create batches to send to groq
import json
def make_jsonl(item):
  body = {'model':'openai/gpt-oss-120b','messages':[{'role':'system',"content":SYSTEM_PROMPT},{"role": "user", "content": str(item),"reasoning_effort":"low"}]}
  line = {"custom_id":str(item.id),"method":"POST","url":"/v1/chat/completions","body":body}
  return json.dumps(line)


make_jsonl(items[0])



'{"custom_id": "0", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "openai/gpt-oss-120b", "messages": [{"role": "system", "content": "Create a concise description of a product. Respond only in this format. Do not include part numbers.\\nTitle: Rewritten short precise title\\nCategory: eg Electronics\\nBrand: Brand name\\nDescription: 1 sentence description\\nDetails: 1 sentence on features"}, {"role": "user", "content": "title=\'Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)\' category=\'Tools_and_Home_Improvement\' price=64.3 full=None weight=1.5 summary=\'Title: Schlage F59 & 613 Andover Interior Knob (Deadbolt Included)  \\\\nCategory: Home Hardware  \\\\nBrand: Schlage  \\\\nDescription: A single\\u2011piece oil\\u2011rubbed bronze knob that mounts to a deadbolt for secure, easy interior door use.  \\\\nDetails: Designed for a 4\\" minimum center\\u2011to\\u2011center door prep, it offers a lifetime mechanical and

In [74]:
MODEL = "openai/gpt-oss-20b"


In [96]:
def make_jsonl(item):
    body = {"model": MODEL, "messages": [{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": str(item)}], "reasoning_effort": "low"}
    line = {"custom_id": str(item.id), "method": "POST", "url": "/v1/chat/completions", "body": body}
    return json.dumps(line)

In [97]:
def make_file(start,end):
  path = "revision/jsonl/"
  with open(file=f"revision/jsonl/{start}_{end}.jsonl",mode='w',encoding='utf-8') as f:
    for i in range(start,end):
      f.write(make_jsonl(items[i]))
      f.write('\n')

make_file(0,1000)


In [98]:
# we need to put our jsonl requests to the cloud so we do groq.files.creater(file) it expects in binary mode
with open(file="revision/jsonl/0_1000.jsonl",mode="rb") as f:
  response = groq.files.create(file=f,purpose="batch")
response

FileCreateResponse(id='file_01kf0v4aprekztgsh799gfjnyq', bytes=1071538, created_at=1768481237, filename='0_1000.jsonl', object='file', purpose='batch', size=0, md5='h9vGFwJSVCJhG1WhY+HAmQ==', content_type='application/jsonl')

In [99]:
# we are setting a timewindow of 24h for groq to reply all our requests
response = groq.batches.create(completion_window='24h',endpoint='/v1/chat/completions',input_file_id=response.id)


In [103]:
# to check whether done or not
result = groq.batches.retrieve(response.id)
print(result)


BatchRetrieveResponse(id='batch_01kf0v4bpcfh5b3tbwtftc4s6b', completion_window='24h', created_at=1768481238, endpoint='/v1/chat/completions', input_file_id='file_01kf0v4aprekztgsh799gfjnyq', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1768481251, error_file_id=None, errors=None, expired_at=None, expires_at=1768567638, failed_at=None, finalizing_at=1768481249, in_progress_at=1768481242, metadata=None, output_file_id='file_01kf0v4psxf8g8s67mb89dcf0g', request_counts=RequestCounts(completed=1000, failed=0, total=1000), project_id='project_01ke9mp2tpfdkbfprz83y5bvq8')


In [104]:
result.status

'completed'

In [105]:
response = groq.files.content(result.output_file_id)

In [108]:
response.write_to_file("revision/jsonl/batch_results.jsonl")

In [111]:
with open("revision/jsonl/batch_results.jsonl",mode="r",encoding='utf-8') as f:
  for line in f:
    json_line = json.loads(line)
    id = int(json_line['custom_id'])
    summary = json_line['response']['body']['choices'][0]['message']['content']
    items[id].summary = summary



In [6]:
print(items[0].summary)

Title: Schlage F59 & 613 Andover Interior Knob (Deadbolt Included)  
Category: Home Hardware  
Brand: Schlage  
Description: A single‑piece oil‑rubbed bronze knob that mounts to a deadbolt for secure, easy interior door use.  
Details: Designed for a 4" minimum center‑to‑center door prep, it offers a lifetime mechanical and finish warranty and comes ready for quick installation.


In [149]:
# Now that we have details reaady we can ask the groq to to guess the prices
SYSTEM_PROMPT="you are a price guesser give the details about a product you need to validate and guess the price of it.strictly reply only the price and we want in dollars no sign required only numbers."


In [150]:
from groq import Groq
# os.environ.get("GROQ_API_KEY")
groq= Groq(api_key=os.getenv('GROQ_API_KEY'))


In [151]:
def make_jsonl(item):
  body = {
    'model' :"openai/gpt-oss-20b",
    'messages':[{'role':'user',"content":item.summary},{'role':'system','content':SYSTEM_PROMPT}],'reasoning_effort':'low'
  }
  line = {'custom_id':str(item.id),'method':'POST','url':'/v1/chat/completions','body':body}
  return json.dumps(line)
make_jsonl(items[0])

'{"custom_id": "0", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "openai/gpt-oss-20b", "messages": [{"role": "user", "content": "Title: Schlage F59 & 613 Andover Interior Knob with Deadbolt, Oil\\u2011Rubbed Bronze  \\nCategory: Tools_and_Home_Improvement  \\nBrand: Schlage  \\nDescription: A single\\u2011piece oil\\u2011rubbed bronze knob that mounts to a deadbolt for secure and easy interior door use.  \\nDetails: Designed for a 4\\" minimum center\\u2011to\\u2011center door prep, it offers lifetime mechanical and finish warranty and comes ready for quick installation."}, {"role": "system", "content": "you are a price guesser give the details about a product you need to validate and guess the price of it.strictly reply only the price and we want in dollars no sign required only numbers."}], "reasoning_effort": "low"}}'

In [7]:
len(items)

22000

In [153]:
with open('requestL/make_request.jsonl',mode='w') as f:
  for item in items[0:1000]:
    f.write(make_jsonl(item))
    f.write('\n')


In [154]:
with open(file = 'requestL/make_request.jsonl',mode="rb") as f:
  response = groq.files.create(file=f,purpose='batch')


In [155]:
response.id

'file_01kf1mcaf0e28sr1rexy0xwaxs'

In [156]:
response = groq.batches.create(input_file_id=response.id,completion_window="24h",endpoint="/v1/chat/completions")


In [160]:
result = groq.batches.retrieve(response.id)
result

BatchRetrieveResponse(id='batch_01kf1mck8qeysb86j6akqwzsfd', completion_window='24h', created_at=1768507723, endpoint='/v1/chat/completions', input_file_id='file_01kf1mcaf0e28sr1rexy0xwaxs', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1768507733, error_file_id=None, errors=None, expired_at=None, expires_at=1768594123, failed_at=None, finalizing_at=1768507731, in_progress_at=1768507726, metadata=None, output_file_id='file_01kf1mcvtxee1v8qsyds50y1qd', request_counts=RequestCounts(completed=1000, failed=0, total=1000), project_id='project_01ke9mp2tpfdkbfprz83y5bvq8')

In [161]:
result.status

'completed'

In [163]:
response = groq.files.content(result.output_file_id)

response.write_to_file('revision/jsonl/batch_results_1000_prices.jsonl')


In [ ]:
y_true=[]
y_pred =np.zeros(1000)


with open(file='revision/jsonl/batch_results_1000_prices.jsonl',mode='r',encoding='utf-8') as f:
  for line in f:
    json_line = json.loads(line)
    id = int(json_line['custom_id'])
    raw = json_line['response']['body']['choices'][0]['message']['content']

    price = float(raw.replace('$', '').replace(',', '').strip())

    y_pred[id] = price



# Front

In [168]:
y_true=[]

for item in items[:1000]:
  y_true.append(item.price)
y_true = np.array(y_true)
rmse = np.sqrt(np.mean((y_true-y_pred)**2))

In [169]:
rmse

np.float64(802.7181207582771)

In [172]:
for i in range(10):
  print(y_pred[i],y_true[i])

45.0 64.3
35.0 79.0
250.0 240.0
290.0 449.0
180.0 79.99
140.0 89.99
75.99 174.99
10.0 74.64
70.0 39.99
39.0 25.0


In [9]:
#Prepare the docs and prices list for vectorization

y = np.array([float(item.price) for item in train])
documents = [item.summary for item in train]

len(documents[:1000])

1000

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
np.random.seed(42)
vectorizer = HashingVectorizer(n_features=5000,stop_words='english',binary=True)
X= vectorizer.fit_transform(documents)


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class NeuralNetwork(nn.Module):
  def __init__(self,input_size):
    super().__init__()
    self.layer1 = nn.Linear(input_size,128);
    self.layer2 = nn.Linear(128,64);
    self.layer3 = nn.Linear(64,64);
    self.layer4 = nn.Linear(64,64);
    self.layer5 = nn.Linear(64,64);
    self.layer6 = nn.Linear(64,64);
    self.layer7= nn.Linear(64,64);
    self.layer8 = nn.Linear(64,1);
    self.relu = nn.ReLU()
    
  def forward(self,x):
    output1 = self.relu(self.layer1(x))
    output2 = self.relu(self.layer2(output1))
    output3 = self.relu(self.layer3(output2))
    output4 = self.relu(self.layer4(output3))
    output5 = self.relu(self.layer5(output4))
    output6 = self.relu(self.layer6(output5))
    output7 = self.relu(self.layer7(output6))
    output8 = self.layer8(output7)
    return output8




In [28]:
X_train_tensor = torch.FloatTensor(X.toarray())
y_train_tensor = torch.FloatTensor(y).unsqueeze(1)
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(X_train_tensor, y_train_tensor, test_size=0.01, random_state=42)
train_dataset = TensorDataset(X_train,y_train)
trainLoader = DataLoader(train_dataset,batch_size=64,shuffle=True)

#Initialize the model
input_size = X_train_tensor.shape[1]
input_size
model = NeuralNetwork(input_size)

In [29]:
# lets analyze the network 
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_params

669249

In [31]:
loss_function = nn.MSELoss()
import torch.optim as optim

optimizer = optim.Adam(model.parameters(),lr=0.01)

EPOCH =  2

for epoch in range(EPOCH):
  model.train() # # sets the model at training mode
  for batch_X,batch_y in tqdm(trainLoader):
    optimizer.zero_grad() # clears the gradient

    outputs = model(batch_X)
    loss = loss_function(outputs,batch_y)
    loss.backward()
    optimizer.step()
  model.eval()

  with torch.no_grad():
    val_outputs = model(X_val)
    val_loss = loss_function(val_outputs,y_val)
  
  print(f'Epoch [{epoch+1}/{EPOCH}], Train Loss: {loss.item():.3f}, Val Loss: {val_loss.item():.3f}')


100%|██████████| 310/310 [00:04<00:00, 65.91it/s]


Epoch [1/2], Train Loss: 27338.307, Val Loss: 15959.360


100%|██████████| 310/310 [00:04<00:00, 72.48it/s]

Epoch [2/2], Train Loss: 15719.445, Val Loss: 18486.736


In [32]:
def neural_network(item):
  model.eval()
  with torch.no_grad():
    vector = vectorizer.fit_transform([item.summary])
    vector = torch.FloatTensor(vector.toarray())
    result = model(vector)[0].item()
  return max(0,result)


evaluate(neural_network,test)


 46%|████▌     | 92/200 [00:00<00:00, 498.09it/s]

$139 $0 $27 $8 $24 $174 $23 $29 $18 $35 $425 $190 $28 $105 $8 $6 $25 $5 $128 $30 $62 $36 $44 $7 $274 $315 $312 $23 $103 $50 $34 $91 $81 $27 $25 $220 $50 $76 $145 $38 $150 $15 $7 $113 $139 $39 $51 $90 $4 $63 $3 $79 $239 $42 $172 $14 $46 $76 $20 $42 $93 $32 $36 $16 $377 $201 $66 $294 $32 $122 $10 $29 $164 $73 $19 $38 $12 $22 $20 $35 $80 $79 $40 $34 $9 $41 $45 $159 $54 $85 $36 $14 $6 $18 $45 $75 $37 $37 $168 $352 $32 $64 $16 $79 $21 $0 $59 $297 $8 $53 $27 $124 $71 $2 $108 $116 $91 $37 $56 $6 $27 $155 $27 $10 $103 $19 $33 $50 $11 $52 $4 $101 $78 $39 $47 $38 $105 $58 $64 $16 $188 $169 $14 $192 $111 $49 $32 $351 $142 $1 $6 $74 $10 $57 $12 $177 $75 $0 

100%|██████████| 200/200 [00:00<00:00, 637.24it/s]


$102 $23 $155 $4 $11 $27 $244 $6 $134 $29 $3 $43 $28 $28 $232 $47 $16 $15 $9 $24 $34 $16 $393 $6 $174 $49 $17 $44 $98 $8 $2 $8 $60 $8 $18 $50 $19 $16 $56 $21 $2 $3 

## Frontier Models

In [33]:
def messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [{"role": "user", "content": message}]

In [35]:
# The function for gpt-4.1-nano
from litellm import completion
def gpt_4__1_nano(item):
    response = completion(model="openai/gpt-4.1-nano", messages=messages_for(item))
    return response.choices[0].message.content

In [36]:
evaluate(gpt_4__1_nano, test)

  0%|          | 1/200 [00:02<07:09,  2.16s/it]

$31 $34 $30 

  2%|▏         | 4/200 [00:08<06:36,  2.02s/it]

$20 $120 $130 $6 $65 $11 

  6%|▋         | 13/200 [00:08<01:26,  2.17it/s]

...

$99 $15 $11 

 98%|█████████▊| 195/200 [00:26<00:00, 10.02it/s]

$20 $70 

100%|██████████| 200/200 [00:26<00:00,  7.54it/s]

$10 $70 $30 $1 

In [37]:
len(test)

1000